In [25]:
from cmath import isnan
import os
from random import random
import time
from typing import Any
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from csv import writer

# read the data from a file
data = pd.read_csv(os.getcwd().replace('code','') + "/data/cleaned_prof_data_v4.csv")

# initialize web driver for selenium
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.implicitly_wait(random() * 10)


def findMaxCitations(web_elements: list):
    indOfMax = 0
    maxVal = 0
    for i in range(len(web_elements)):
        val = int(web_elements[i].text.split(" - ")[2].split(" ")[2])
        if val > maxVal:
            maxVal = val
            indOfMax = i
        else:
            continue
    return web_elements[indOfMax]


# loop through and find all professors with wrong / no google scholar ID
professors = []
professorExists = True

for i in range(11137, 12000):
    
    # initialize values
    scholar_id = data.iat[i, 3]
    citation = data.iat[i, 4]
    h_index = data.iat[i, 5]

    if (data.iat[i, 3] == "NOSCHOLARPAGE" or data.iat[i, 3] == "#NAME?" or isnan(data.iat[i, 4]) or isnan(data.iat[i, 5])):
        # launch URL
        driver.get("https://scholar.google.com")

        # identify search box
        box = driver.find_element(By.NAME, "q")

        # enter search text
        box.send_keys(data.iat[i, 0] + " " + data.iat[i, 1])

        # make search
        box.send_keys(Keys.ENTER)

        # in the case that the author pops up as a recommended option
        try:
            link = driver.find_element(
                By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]/table/tbody/tr/td[2]/h4/a")
            link.click()
            time.sleep(random() * 3)
            
        except:
            try:
                # in the case there are multiple options in the recommended table
                list_of_elements = driver.find_element(
                    By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]/table/tbody/tr/td[2]").find_elements(By.TAG_NAME, "div")
                max_element = findMaxCitations(list_of_elements)
                max_element.find_element(By.TAG_NAME, "a").click()
                time.sleep(random() * 5)
            except:
                # otherwise loop through all the divs and find the correct professor
                elements_list = driver.find_element(
                    By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]").find_element(By.CLASS_NAME, "gs_a").find_elements(By.TAG_NAME, "a")
                if elements_list: 
                    for element in elements_list:  # looping through the number of professors in the link
                        try:
                            element.find_element(By.TAG_NAME, "b")
                            element.click()
                            professorExists = True
                            break
                        except NoSuchElementException:
                            professorExists = False
                            continue  # in the case that the element is not found
                    # give time for html / css / js to load
                    time.sleep(random() * 5)
                else: 
                    professorExists = False 

        # retrieve citation, h-index, and scholar_id for that prof
        if professorExists:
            scholar_id = driver.current_url.split("=")[1].split("&")[0]
            citation = int(driver.find_element(
                By.XPATH, "//*[@id='gsc_rsb_st']/tbody/tr[1]/td[2]").text)
            h_index = int(driver.find_element(
                By.XPATH, "//*[@id='gsc_rsb_st']/tbody/tr[2]/td[2]").text)
            time.sleep(random() * 5)
        else:
            scholar_id = None
            citation = None
            h_index = None
    
    print(i)
    professor = { 
        "Name": data.iat[i, 0],
        "affiliation": data.iat[i, 1],
        "homepage": data.iat[i, 2],
        "scholar-id": scholar_id,
        "citations": citation,
        "h-index": h_index
    }
    print(professor)
    professors.append(professor)


df = pd.DataFrame(professors)
df.to_csv(os.getcwd().replace('code','') +"data/data5.csv")


[WDM] - ====== WebDriver manager ======


2022-07-28 08:47:15,486 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-28 08:47:15,582 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


...

11790
{'Name': 'Young-ri Choi', 'affiliation': 'UNIST', 'homepage': 'http://ychoi.unist.ac.kr', 'scholar-id': 'veZ38FUAAAAJ', 'citations': 2645, 'h-index': 15}
11791
{'Name': 'Seungryul Baek', 'affiliation': 'UNIST', 'homepage': 'https://sites.google.com/site/bsrvision00', 'scholar-id': 'Mz1fab8AAAAJ', 'citations': 662.0, 'h-index': 9.0}
11792
{'Name': 'Kwang In Kim', 'affiliation': 'UNIST', 'homepage': 'https://researchportal.bath.ac.uk/en/persons/kwang-in-kim', 'scholar-id': '0wIdMGEAAAAJ', 'citations': 7567.0, 'h-index': 31.0}
11793
{'Name': 'Jaeyeon Lee', 'affiliation': 'UNIST', 'homepage': 'https://jaeyeonlee.com', 'scholar-id': 'K_xmrzgAAAAJ', 'citations': 241.0, 'h-index': 7.0}
11794
{'Name': 'Chiehyeon Lim', 'affiliation': 'UNIST', 'homepage': 'http://service.unist.ac.kr', 'scholar-id': 'NW7F08MAAAAJ', 'citations': 1619.0, 'h-index': 19.0}
11795
{'Name': 'Gi-Soo Kim', 'affiliation': 'UNIST', 'homepage': 'https://sdm.unist.ac.kr', 'scholar-id': 'uW2qIcYAAAAJ', 'citations': 118.0

11838
{'Name': 'Bruno A. Gaëta', 'affiliation': 'UNSW', 'homepage': 'https://research.unsw.edu.au/people/dr-bruno-andre-gaeta', 'scholar-id': '7BDBTt0AAAAJ', 'citations': 2108, 'h-index': 21}
11839
{'Name': 'Abdallah Saffidine', 'affiliation': 'UNSW', 'homepage': 'http://www.cse.unsw.edu.au/~abdallahs', 'scholar-id': 'yWzT7LAAAAAJ', 'citations': 780.0, 'h-index': 16.0}
11840
{'Name': 'Hailun Tan', 'affiliation': 'UNSW', 'homepage': 'https://www.engineering.unsw.edu.au/computer-science-engineering/staff/hailun-tan', 'scholar-id': 'Vh9SrzQAAAAJ', 'citations': 398.0, 'h-index': 9.0}
11841
{'Name': 'Michael Bain', 'affiliation': 'UNSW', 'homepage': 'https://research.unsw.edu.au/people/dr-michael-bain', 'scholar-id': 'Jje4QFQAAAAJ', 'citations': 2429, 'h-index': 25}
11842
{'Name': 'Nadine Marcus', 'affiliation': 'UNSW', 'homepage': 'https://research.unsw.edu.au/people/dr-nadine-marcus', 'scholar-id': 'EFmKXMwAAAAJ', 'citations': 2529, 'h-index': 19}
11843
{'Name': 'Haojun Ma', 'affiliation'

11888
{'Name': 'Yang Wang', 'affiliation': 'UNSW', 'homepage': 'http://www.cse.unsw.edu.au/~wangy', 'scholar-id': 'cMSEByAAAAAJ', 'citations': 933, 'h-index': 16}
11889
{'Name': 'Junior Barrera', 'affiliation': 'USP', 'homepage': 'http://www.ime.usp.br/~jb', 'scholar-id': 'FSrFOAMAAAAJ', 'citations': 2750, 'h-index': 25}
11890
{'Name': 'Nina S. T. Hirata', 'affiliation': 'USP', 'homepage': 'http://www.ime.usp.br/~nina', 'scholar-id': 'HnC89_0AAAAJ', 'citations': 1298, 'h-index': 18}
11891
{'Name': 'Julio Michael Stern', 'affiliation': 'USP', 'homepage': 'http://www.ime.usp.br/~jstern', 'scholar-id': '7spXyx8AAAAJ', 'citations': 1841.0, 'h-index': 23.0}
11892
{'Name': 'Kelly Rosa Braghetto', 'affiliation': 'USP', 'homepage': 'http://www.ime.usp.br/~kellyrb', 'scholar-id': 'GZ66SBcAAAAJ', 'citations': 326.0, 'h-index': 11.0}
11893
{'Name': 'Fabio Kon', 'affiliation': 'USP', 'homepage': 'http://www.ime.usp.br/~kon', 'scholar-id': 'VE8BW84AAAAJ', 'citations': 6766.0, 'h-index': 40.0}
11894

11944
{'Name': 'André C. P. L. F. Carvalho', 'affiliation': 'USP-ICMC', 'homepage': 'http://www.cemeai.icmc.usp.br/andre', 'scholar-id': None, 'citations': None, 'h-index': None}
11945
{'Name': 'Simone R. S. Souza', 'affiliation': 'USP-ICMC', 'homepage': 'http://icmc.usp.br/pessoas?id=3685313', 'scholar-id': 'EhrqxfYAAAAJ', 'citations': 1265.0, 'h-index': 18.0}
11946
{'Name': 'Seiji Isotani', 'affiliation': 'USP-ICMC', 'homepage': 'http://icmc.usp.br/pessoas?id=6064889', 'scholar-id': 'GKphfGsAAAAJ', 'citations': 5686.0, 'h-index': 37.0}
11947
{'Name': 'Alexandre C. B. Delbem', 'affiliation': 'USP-ICMC', 'homepage': 'http://icmc.usp.br/pessoas?id=470689', 'scholar-id': 'LYsCAa4AAAAJ', 'citations': 3138.0, 'h-index': 27.0}
11948
{'Name': 'Júlio Cezar Estrella', 'affiliation': 'USP-ICMC', 'homepage': 'http://icmc.usp.br/pessoas?id=10112875', 'scholar-id': 'uDdnClUAAAAJ', 'citations': 1336.0, 'h-index': 16.0}
11949
{'Name': 'Vanderlei Bonato', 'affiliation': 'USP-ICMC', 'homepage': 'http:

11989
{'Name': 'Yinlong Xu', 'affiliation': 'USTC', 'homepage': 'http://en.nhpcc.ustc.edu.cn/Peoples_9/201107/t20110701_115017.html', 'scholar-id': None, 'citations': None, 'h-index': None}
11990
{'Name': 'Yan Xiong', 'affiliation': 'USTC', 'homepage': 'http://dsxt.ustc.edu.cn/zj_ywjs.asp?zzid=709', 'scholar-id': 'X08c_CQAAAAJ', 'citations': 4329.0, 'h-index': 24.0}
11991
{'Name': 'Yinxing Xue', 'affiliation': 'USTC', 'homepage': 'http://dsxt.ustc.edu.cn/zj_js.asp?zzid=6495', 'scholar-id': None, 'citations': None, 'h-index': None}
11992
{'Name': 'Guangzhong Sun', 'affiliation': 'USTC', 'homepage': 'http://staff.ustc.edu.cn/~gzsun', 'scholar-id': 'sm_uYE8AAAAJ', 'citations': 5573.0, 'h-index': 24.0}
11993
{'Name': 'Xiaoping Chen', 'affiliation': 'USTC', 'homepage': 'http://dsxt.ustc.edu.cn/zj_ywjs.asp?zzid=471', 'scholar-id': '7_dDukkAAAAJ', 'citations': 3217.0, 'h-index': 19.0}
11994
{'Name': 'Panlong Yang', 'affiliation': 'USTC', 'homepage': 'http://cs.ustc.edu.cn/szdw/bdjs/201707/t20

In [17]:
professors

[{'Name': 'Abdul Serwadda',
  'affiliation': 'Texas Tech University',
  'homepage': 'http://www.depts.ttu.edu/CS/faculty/faculty.php?name=Abdul%20Serwadda',
  'scholar-id': '0YfFzzUAAAAJ',
  'citations': 1025.0,
  'h-index': 16.0},
 {'Name': 'Susan A. Mengel',
  'affiliation': 'Texas Tech University',
  'homepage': 'http://www.depts.ttu.edu/CS/faculty/faculty.php?name=Susan%20A.%20Mengel',
  'scholar-id': None,
  'citations': None,
  'h-index': None},
 {'Name': 'Richard Watson',
  'affiliation': 'Texas Tech University',
  'homepage': 'http://www.depts.ttu.edu/cs/faculty/faculty.php?name=Richard%20Watson',
  'scholar-id': None,
  'citations': None,
  'h-index': None},
 {'Name': 'Yu Zhuang',
  'affiliation': 'Texas Tech University',
  'homepage': 'http://www.depts.ttu.edu/cs/faculty/faculty.php?name=Yu%20Zhuang',
  'scholar-id': None,
  'citations': None,
  'h-index': None},
 {'Name': 'Dang Tuan Nhon',
  'affiliation': 'Texas Tech University',
  'homepage': 'http://www.myweb.ttu.edu/tnho

In [18]:
df = pd.DataFrame(professors)
df.to_csv(os.getcwd().replace('code','') +"data/data4.csv")

In [42]:
data.loc[data['name']== 'Shangfei Wang']

,name,affiliation,homepage,scholar_id,citations,h-index
11999,Shangfei Wang,USTC,http://staff.ustc.edu.cn/~sfwang,PmDOUfQAAAAJ,3400.0,27.0


In [31]:
big_data = pd.DataFrame()
for i in range(5):
    temp_data = pd.read_csv(os.getcwd().replace('code','') + f'/data/data{i+1}.csv')
    temp_data = temp_data.drop(['Unnamed: 0'], axis = 1)
    big_data = pd.concat([big_data,temp_data])
big_data.to_csv(os.getcwd().replace('code','') + '/data/cleaned_prof_data_v5_2.csv', index = False)

In [44]:
data1 = pd.read_csv(os.getcwd().replace('code','') + "/data/cleaned_prof_data_8000_12000.csv")
data2 = pd.read_csv(os.getcwd().replace('code','') + "/data/replaced_bad_prof_data_12000_20650.csv")

In [47]:
data_3 = pd.concat([data1,data2]).reset_index(drop=True)
data_3 = data_3.drop(['Unnamed: 0'], axis = 1)
data_3.to_csv(os.getcwd().replace('code','') + "/data/replaced_data_8000_20650.csv", index = False)

In [48]:
data4 = pd.read_csv(os.getcwd().replace('code','') + "/data/replaced_data_8000_20650.csv")



In [50]:
data4.loc[pd.isna(data['h-index']) & ~pd.isna(data4['h-index'])]

,Name,affiliation,homepage,scholar-id,citations,h-index,Unnamed: 0
0,Liangcai Gao,Peking University,http://www.icst.pku.edu.cn/cpdp/index.php/rese...,6Ia92WsAAAAJ,1387.0,22.0,NaN
13,Shiliang Zhang,Peking University,http://idm.pku.edu.cn/staff/zhangshiliang/inde...,7phvKK4AAAAJ,6866.0,34.0,NaN
38,Wei Ye,Peking University,https://se.pku.edu.cn/kcl/weiye,RgLGFMIAAAAJ,335.0,10.0,NaN
59,Leye Wang,Peking University,http://cs.pku.edu.cn/info/1174/2334.htm,o2I4sL8AAAAJ,4075.0,33.0,NaN
62,Song Chun Zhu,Peking University,http://www.stat.ucla.edu/~sczhu,Al8dyb4AAAAJ,30710.0,86.0,NaN
...,...,...,...,...,...,...,...
6025,Yuanxiang Li,Wuhan University,http://cs.whu.edu.cn/teacherinfo.aspx?id=222,kKt3SNwAAAAJ,115.0,5.0,2026.0
6055,George Tourlakis,York University,http://www.cse.yorku.ca/~gt,ubosF3IAAAAJ,326.0,8.0,2056.0
6090,Deren Chen,Zhejiang University,http://mypage.zju.edu.cn/0073012,qZTMyzwAAAAJ,1473.0,20.0,2091.0
6091,Shijian Li,Zhejiang University,http://mypage.zju.edu.cn/lishijian,7uXF9ZIAAAAJ,5785.0,34.0,2092.0
